In [24]:
!python -m pip install elasticsearch
!python -m pip install eland

     |████████████████████████████████| 204kB 13.4MB/s 


In [25]:
from urllib.request import urlopen
import pandas as pd
import eland as ed
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from elasticsearch import Elasticsearch
# Import standard test settings for consistent results
from eland.conftest import *

# Loading code <utils.py> from github repository.
url_code_github = "https://raw.githubusercontent.com/hoat23/ElasticSearch/master/bin/utils.py"
code_str = urlopen(url_code_github).read()
code_str = code_str.decode('utf-8')
exec(  code_str  )

# Loading code <csv> from github repository
url_data_csv = "https://raw.githubusercontent.com/hoat23/DataAnalytics/master/data/state-population.csv"

In [44]:
# Connect to an Elastic Cloud instance

#es = Elasticsearch(
#  cloud_id=ELASTIC_CLOUD_ID,
#  http_auth=(ELASTIC_USER, ELASTIC_CLOUD_PASSWORD)
#)

es = Elasticsearch(
    [CLUSTER_ENDPOINT],
    api_key=( ID , API_KEY),
)

print_json(es.info())

{
  "cluster_name": "8eb8a6a47de443989c55b121f2d5c033",
  "cluster_uuid": "CpKu9OxRQ7uDSiqLoNGVeg",
  "name": "instance-0000000000",
  "tagline": "You Know, for Search",
  "version": {
    "build_date": "2020-11-09T21:30:33.964949Z",
    "build_flavor": "default",
    "build_hash": "51e9d6f22758d0374a0f3f5c6e8f3a7997850f96",
    "build_snapshot": false,
    "build_type": "docker",
    "lucene_version": "8.7.0",
    "minimum_index_compatibility_version": "6.0.0-beta1",
    "minimum_wire_compatibility_version": "6.8.0",
    "number": "7.10.0"
  }
}


In [51]:
df_pandas = pd.read_csv(url_data_csv).dropna()
# Rename the columns name
df_pandas = df.rename( columns = {"state/region":"state-region"} )
df_pandas.info()
df_pandas.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2524 entries, 0 to 2543
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   state-region  2524 non-null   object 
 1   ages          2524 non-null   object 
 2   year          2524 non-null   int64  
 3   population    2524 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 98.6+ KB


,state-region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0


In [47]:
df_eland = ed.pandas_to_eland(
    pd_df=df_pandas,
    es_client=es,
    # Where the data will live in Elasticsearch
    es_dest_index="state-population",
    # Type overrides for certain columns
    es_type_overrides={
        "state-region": "keyword",
        "ages": "keyword",
        "year": "keyword",
        "population": "double"
    },
    # If the index already exists what should we do?
    es_if_exists="replace",
    # Wait for data to be indexed before returning
    es_refresh=True,
)
df_eland.info()

<class 'eland.dataframe.DataFrame'>
Index: 2524 entries, 2520 to 2519
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ages          2524 non-null   object 
 1   population    2524 non-null   float64
 2   state-region  2524 non-null   object 
 3   year          2524 non-null   object 
dtypes: float64(1), object(3)
memory usage: 80.0 bytes


In [53]:
df_eland.head()

,ages,population,state-region,year
2520,total,287625193.0,USA,2002
2521,under18,72936457.0,USA,2002
2522,total,290107933.0,USA,2003
2523,under18,73100758.0,USA,2003
2524,total,292805298.0,USA,2004


In [60]:
#Filter dataframe u
df_eland.es_query({"match": {"state-region": "USA"}}).head()

,ages,population,state-region,year
2520,total,287625193.0,USA,2002
2521,under18,72936457.0,USA,2002
2522,total,290107933.0,USA,2003
2523,under18,73100758.0,USA,2003
2524,total,292805298.0,USA,2004


In [64]:
df_pd = df_eland.to_pandas()
df_pd.head()

,ages,population,state-region,year
2520,total,287625193.0,USA,2002
2521,under18,72936457.0,USA,2002
2522,total,290107933.0,USA,2003
2523,under18,73100758.0,USA,2003
2524,total,292805298.0,USA,2004
